# 5 합성곱 신경망

모델을 학습하기 위해서는 지속적으로 데이터를 모델에 전달해야 합니다.</br>
</br>
데이터를 하나씩 전달하지 않고 원하는 배치 사이즈대로 묶어서 전달하거나</br>
데이터를 어떤 규칙에 따라 정렬하거나 섞어주는 역할을 해주는 것이 DataLoader 모듈입니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [2]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(),
    target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(),
    target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
    shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
    shuffle=False, num_workers=2, drop_last=True)

100%|██████████| 9.91M/9.91M [00:12<00:00, 821kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 143kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 768kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.54MB/s]


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()    ## init 함수 안의 super 클래스는 부모 클래스인 nn.Module을 초기화하는 역할
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size,-1)   # -1은 -1인 부분을 알아서 계산하라는 의미
        out = self.fc_layer(out)
        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)